In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

Passos desse notebook

1. Merge de datasets

2. Transformar colunas

- Variáveis Categóricas (Partido, Tema, RegimeTramitacao, Ano)
    - One-Hot Encoding
- Variáveis Numéricas (Quantidade de votos do partido X)
    - Normalizar

In [15]:
ListaProposicoes = os.path.join('data', 'ListaProposicoes_2013_2024.csv')
ListaProposicoesVotadas_Plenario = os.path.join('data', 'ListaProposicoesVotadasPlenario_2013_2024.csv')
ObterProposicoes = os.path.join('data', 'ObterProposicoes_2013_2024.csv')
ObterProposicoes_2013 = os.path.join('data', 'obter_proposicoes' ,'ObterProposicoes_2013.csv')
VotacoesProposicoes = os.path.join('data', 'VotacoesProposicoes_2013_2024.csv')

In [9]:
df = pd.read_csv(ObterProposicoes)
df_pvp = pd.read_csv(ListaProposicoesVotadas_Plenario)
df_vp = pd.read_csv(VotacoesProposicoes)
df_lp = pd.read_csv(ListaProposicoes)

In [10]:
print("Quantidade de linhas em df: ", df.shape[0])
print("Quantidade de linhas em df_lp: ", df_lp.shape[0])  

print("Quantidade de colunas em df: ", df.shape[1])
print("Quantidade de colunas em df_lp: ", df_lp.shape[1]) 

Quantidade de linhas em df:  132399
Quantidade de linhas em df_lp:  39701
Quantidade de colunas em df:  21
Quantidade de colunas em df_lp:  31


In [13]:
df.idProposicao.nunique()

39701

In [21]:
# Imprimir valores duplicados na coluna 'idProposicao'
duplicated_ids = df[df['idProposicao'].duplicated()]['idProposicao']
print(duplicated_ids)

12832     605259
12833     605279
12834     605290
12835     605291
12836     605307
           ...  
132394    574182
132395    569556
132396    568561
132397    568597
132398    565698
Name: idProposicao, Length: 92698, dtype: int64


## Removendo duplicatas

Devido ao processamento paralelo das requisições, alguns dados foram salvos mais de uma vez com base no ID. Para garantir a integridade dos dados, as duplicatas serão eliminadas.


In [25]:
duplicated_ids = df[df['idProposicao'].duplicated()]['idProposicao']
#print(duplicated_ids)

if duplicated_ids.any():
    df = df.drop_duplicates(subset='idProposicao')

#print(df)

In [11]:
df.columns

Index(['nomeProposicao', 'idProposicao', 'idProposicaoPrincipal',
       'nomeProposicaoOrigem', 'tipoProposicao', 'tema', 'Ementa',
       'ExplicacaoEmenta', 'Autor', 'ideCadastro', 'ufAutor', 'partidoAutor',
       'DataApresentacao', 'RegimeTramitacao', 'UltimoDespacho', 'Apreciacao',
       'Indexacao', 'Situacao', 'LinkInteiroTeor', 'apensadas',
       'apensadas_proposicao'],
      dtype='object')

In [ ]:
df_tran = df[df.Situacao.str.contains('Tran')]
df_tran

In [47]:
df['DataApresentacao'] = pd.to_datetime(df['DataApresentacao'], format='%d/%m/%Y')

/tmp/ipykernel_101418/3532351135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DataApresentacao'] = pd.to_datetime(df['DataApresentacao'], format='%d/%m/%Y')


In [48]:
df.DataApresentacao.value_counts().sort_index()

DataApresentacao
2013-02-04    13
2013-02-05    20
2013-02-06    11
2013-02-07     5
2013-02-08     1
              ..
2024-10-09    14
2024-10-10    33
2024-10-11    12
2024-10-14    14
2024-10-15    24
Name: count, Length: 2344, dtype: int64

dv_vp

Tem algo errado.


In [51]:
df_vp.nomeproposicao.value_counts()

nomeproposicao
PL 3729/2004                                 2772
PL 8939/2017                                 2700
PL 3261/2019                                 2592
PL 5029/2019 (Nº Anterior: PL 11021/2018)    2171
PL 5735/2013                                 2140
                                             ... 
PL 1847/2024                                   11
PL 3/2024                                       9
PL 858/2024                                     8
PL 1213/2024                                    5
PL 3117/2024                                    4
Name: count, Length: 243, dtype: int64

In [54]:
df_vp

,resumo,data,hora,objvotacao,codsessao,nomeproposicao,bancada_sigla,bancada_orientacao,deputado_nome,deputado_idecadastro,deputado_partido,deputado_uf,deputado_voto
0,Mantido o texto. Sim: 268; não: 62; abstenção:...,9/7/2013,23:02,DVS - DEM - § 2º DO ART. 23 DO SUBSTITUTIVO DO...,4976,PL 4529/2004,Minoria,Liberado,Pedro Guerra,174314,PSD,PR,Sim
1,Mantido o texto. Sim: 268; não: 62; abstenção:...,9/7/2013,23:02,DVS - DEM - § 2º DO ART. 23 DO SUBSTITUTIVO DO...,4976,PL 4529/2004,Minoria,Liberado,Pedro Guerra,174314,PSD,PR,Sim
2,Rejeitado o Requerimento. Sim: 2; não: 270; ab...,5/11/2013,20:53,REQUERIMENTO DE RETIRADA DE PAUTA,5097,PL 6025/2005,Repr.PMN,Não,Amir Lando,174861,PMDB,RO,Não
3,Mantido o parágrafo. Sim: 206; não: 159; abste...,4/2/2014,21:30,DVS - PP - § 19 DO ART. 85 DA EMENDA AGLUTINATIVA,5151,PL 6025/2005,GOV.,Não,Helcio Silva,175737,PT,SP,Não
4,Aprovada a Emenda Aglutinativa nº 7. Sim: 279;...,11/2/2014,20:57,DVS - PTB - EMENDA AGLUTINATIVA Nº 7,5160,PL 6025/2005,GOV.,Não,Helcio Silva,175737,PT,SP,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104181,Mantido o texto. Sim: 206; Não: 70; Total: 276.,11/9/2024,00:26,DTQ. 4 - PL - ARTIGO 43.,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104182,Mantido o texto. Sim: 200; Não: 80; Abstenção:...,11/9/2024,00:36,DTQ. 2 - PL - ARTIGO 44.,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104183,Mantido o texto. Sim: 189; Não: 140; Total: 329.,11/9/2024,00:55,DTQ. 3 - PL - CAPÍTULO VIII,17505,PL 1847/2024,Governo,Sim,Newton Bonin,229939,União,PR,Sim
104184,Rejeitada a Emenda de Plenário nº 18. Sim: 133...,11/9/2024,00:13,DTQ. 5 - PL - EMENDA DE PLENÁRIO Nº 18,17505,PL 1847/2024,Governo,Não,Newton Bonin,229939,União,PR,Não


Colunas selecionadas para ficar:

In [37]:
df = df[["nomeProposicao", "idProposicao", "tema", "partidoAutor", "DataApresentacao", "RegimeTramitacao","Situacao"]]